# Solución Caso 4


## Conjuntos Necesarios


In [4]:
import pandas as pd
#Clientes y sus datos
clients_df = pd.read_csv('data/Clients.csv')
clients_dict = {f"C{int(row['ClientID'])}": [int(row['Product-Type-A']), int(row['Product-Type-B']), int(row['Product-Type-C']),row['Longitude'],row['Latitude']] for _, row in clients_df.iterrows()}
print(clients_dict)

#Depositos y sus datos
deposits_df = pd.read_csv('data/Depots_With_Products.csv')
deposits_dict = {f"D{int(row['DepotID'])}": [int(row['DepotID']),row['Longitude'],row['Latitude'], int(row['Product-Type-A']), int(row['Product-Type-B']), int(row['Product-Type-C'])] for _, row in deposits_df.iterrows()}
print(deposits_dict)

#Vehículos y sus datos
vehicles_df = pd.read_csv('data/Vehicles.csv')
type_mapping = {
    'gas car': 'GS',
    'ev': 'EV',
    'solar ev': 'EV',
    'drone': 'DR'
}
vehicles_dict = {}
for idx, row in enumerate(vehicles_df.itertuples(), start=1):
    vehicle_id = f"V{idx}"  # Claves como V1, V2, ...
    vehicles_dict[vehicle_id] = [type_mapping[row.VehicleType.lower()], row.Capacity, row.Range]
print(vehicles_dict)

#Datos de Costos por tipo de vehículo
vehicles_data_df = pd.read_csv('data/vehicles_data.csv')
vehicles_data_df.rename(
    columns={
        'Vehicle': 'Type',
        'Freight Rate [COP/km]': 'TF',
        'Time Rate [COP/min]': 'TT',
        'Daily Maintenance [COP/day]': 'CM',
        'Recharge/Fuel Cost [COP/(gal or kWh)]': 'CRC',
        'Recharge/Fuel Time [min/10 percent charge]': 'TR',
        'Avg. Speed [km/h]': 'VP',
        'Gas Efficiency [km/gal]': 'Gas_Efficiency',
        'Electricity Efficency [kWh/km]': 'Electricity_Efficiency'
    },
    inplace=True
)

vehicles_parameters = {}
for _, row in vehicles_data_df.iterrows():
    vehicle_type = type_mapping[row['Type'].lower()]
    vehicles_parameters[vehicle_type] = {
        'TF': row['TF'],
        'TT': row['TT'],
        'CM': row['CM'],
        'CRC': row['CRC'],
        'TR': row['TR'],
        'VP': row['VP'],
        'EC': row['Gas_Efficiency'] if not pd.isna(row['Gas_Efficiency']) else row['Electricity_Efficiency']
    }
print(vehicles_parameters)

#Matriz de distancias terrestres
distance_matrix_df = pd.read_csv('data/distance_matrix.csv', index_col=0)
# distance_matrix_df = pd.read_csv('distance_single_matrix.csv', index_col=0)
distance_dict = {
    (row, col): float(distance_matrix_df.loc[row, col])
    for row in distance_matrix_df.index
    for col in distance_matrix_df.columns
}
print({key: distance_dict[key] for key in list(distance_dict.keys())[:20]})

#Matriz de tiempos terrestres
duration_matrix_df = pd.read_csv('data/duration_matrix.csv', index_col=0)
# duration_matrix_df = pd.read_csv('duration_single_matrix.csv', index_col=0)
duration_dict = {
    (row, col): float(duration_matrix_df.loc[row, col])
    for row in duration_matrix_df.index
    for col in duration_matrix_df.columns
}
print({key: duration_dict[key] for key in list(duration_dict.keys())[:20]})

#Matriz de distancias aéreas
distance_air_df = pd.read_csv('data/distance_dron_matrix.csv', index_col=0)
# distance_air_df = pd.read_csv('distance_dron_single_matrix.csv', index_col=0)
distance_air_dict = {
    (row, col): float(distance_air_df.loc[row, col])
    for row in distance_air_df.index
    for col in distance_air_df.columns
}
print({key: distance_air_dict[key] for key in list(distance_air_dict.keys())[:20]})

{'C1': [15, 3, 1, -74.19699184741948, 4.632552840424734], 'C2': [9, 1, 3, -74.12032773611021, 4.719616298449461], 'C3': [15, 2, 1, -74.11027242131048, 4.727691608175209], 'C4': [7, 3, 4, -74.06815337963502, 4.712202539809157], 'C5': [12, 1, 4, -74.19486224157397, 4.638612189685843], 'C6': [14, 1, 1, -74.12751619352659, 4.630489482968551], 'C7': [5, 2, 4, -74.10178731827096, 4.732421040989568], 'C8': [15, 1, 1, -74.0328033058973, 4.703342234821396], 'C9': [7, 1, 4, -74.08665709611086, 4.574297477559144]}
{'D1': [1, -74.08124218159384, 4.75021190869025, 140, 20, 50], 'D2': [2, -74.10993358606953, 4.5363832206427785, 160, 0, 20], 'D3': [3, -74.03854814565923, 4.792925960208614, 240, 20, 30], 'D4': [4, -74.06706883098641, 4.72167778077445, 160, 50, 10], 'D5': [5, -74.13826337931849, 4.607707046760958, 70, 0, 40], 'D6': [6, -74.12400186370824, 4.650463053612691, 140, 50, 80], 'D7': [7, -74.09561875464892, 4.621911772492814, 170, 60, 20], 'D8': [8, -74.10975623736951, 4.678960680833056, 30, 

## Modelo


In [2]:
from pyomo.environ import *

model = ConcreteModel()

# Conjuntos
unique_nodes = set(i for i, j in distance_dict.keys()).union(j for i, j in distance_dict.keys())
model.I = Set(initialize=unique_nodes)  # Nodos (orígenes y destinos)
model.PAIRS = Set(dimen=2, initialize=distance_dict.keys())  # Pares válidos (i, j)
model.K = Set(initialize=vehicles_dict.keys())  # Vehículos
model.D = Set(initialize=deposits_dict.keys())  # Depósitos
model.C = Set(initialize=clients_dict.keys())  # Clientes
model.P = Set(initialize=['A','B','C'])  # Tipos de productos

# Parámetros distancia, tiempo, etc.
model.distance = Param(model.PAIRS, initialize=distance_dict)
model.duration = Param(model.PAIRS, initialize=duration_dict)
model.distance_air = Param(model.PAIRS, initialize=distance_air_dict)

model.TF = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TF'] for k in model.K})
model.TT = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TT'] for k in model.K})
model.CM = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['CM'] for k in model.K})
model.CRC = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['CRC'] for k in model.K})
model.TR = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TR'] for k in model.K})
model.VP = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['VP'] for k in model.K})
model.EC = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['EC'] for k in model.K})
model.R = Param(model.K, initialize={k: vehicles_dict[k][2] for k in model.K})

# Parámetros de demanda de clientes por producto
def client_demand_init(model, c, p):
    p_index = {'A':0, 'B':1, 'C':2}[p]
    return clients_dict[c][p_index]  # Demanda del producto p en cliente c
model.demand = Param(model.C, model.P, initialize=client_demand_init, default=0)

# Parámetros de capacidad de depósito por producto
def depot_capacity_init(model, d, p):
    p_index = {'A':3, 'B':4, 'C':5}[p]
    return deposits_dict[d][p_index]
model.depot_capacity = Param(model.D, model.P, initialize=depot_capacity_init, default=0)

# Capacidad máxima del vehículo (total, no por producto) 
# Si se requiere por producto, ajustar
model.capacity_veh = Param(model.K, initialize={k: vehicles_dict[k][1] for k in model.K})

# Variables
model.x = Var(model.I, model.I, model.K, domain=Binary)
model.u = Var(model.I, model.K, domain=NonNegativeReals)

# Cantidad transportada por vehículo k de producto p
model.W = Var(model.K, model.P, domain=NonNegativeReals)

# Variables auxiliares para linealizar entrega desde depósito
model.s = Var(model.K, model.D, domain=Binary)
model.W_s = Var(model.K, model.D, model.P, domain=NonNegativeReals)

# Función objetivo
def total_cost_rule(model):
    C_total = sum(
        model.TF[k] * ((model.distance[i,j]/1000) if vehicles_dict[k][0] != 'DR' else model.distance_air[i,j]) * model.x[i,j,k]
        for k in model.K for (i,j) in model.PAIRS if i!=j
    )

    C_tiempo = sum(
        model.TT[k] * (
            (model.distance[i,j]/1000/(model.VP[k]/60)) if vehicles_dict[k][0] == 'DR' else (model.duration[i,j]/60)
        ) * model.x[i,j,k]
        for k in model.K for (i,j) in model.PAIRS if i!=j
    )

    # Costo por peso (ahora sumando todos los productos)
    C_carga = sum(model.W[k,p]*500 for k in model.K for p in model.P)

    C_mantenimiento = sum(
        model.CM[k]*sum(model.x[d,j,k] for d in model.D for j in model.I if d!=j)
        for k in model.K
    )

    return C_total + C_tiempo + C_carga + C_mantenimiento

model.obj = Objective(rule=total_cost_rule, sense=minimize)

# Restricciones
# Salida desde un depósito
def salida_desde_deposito_rule(model, k):
    return sum(model.x[d, j, k] for d in deposits_dict.keys() for j in model.I if d != j) <= 1
model.salida_desde_deposito = Constraint(model.K, rule=salida_desde_deposito_rule)

# Regreso al depósito de origen
def regreso_a_deposito_rule(model, k):
    return sum(model.x[j, d, k] for d in deposits_dict.keys() for j in model.I if d != j) == sum(model.x[d, j, k] for d in deposits_dict.keys() for j in model.I if d != j)
model.regreso_a_deposito = Constraint(model.K, rule=regreso_a_deposito_rule)

def flujo_correcto_rule(model, i, k):
    if i in deposits_dict.keys():  # No aplica para depósitos
        return Constraint.Skip
    return sum(model.x[i, j, k] for j in model.I if i != j) == sum(model.x[j, i, k] for j in model.I if i != j)
model.flujo_correcto = Constraint(model.I, model.K, rule=flujo_correcto_rule)

def mtz_rule(model, i, j, k):
    if i == j or i in deposits_dict.keys() or j in deposits_dict.keys():
        return Constraint.Skip
    n = len(model.I)
    return model.u[i, k] - model.u[j, k] + (n - 1) * model.x[i, j, k] <= n - 2
model.mtz = Constraint(model.I, model.I, model.K, rule=mtz_rule)

# Capacidad total del vehículo
def capacidad_carga_rule(model, k):
    return sum(model.W[k,p] for p in model.P) <= model.capacity_veh[k]
model.capacidad_carga = Constraint(model.K, rule=capacidad_carga_rule)


# Ajustar si se requiere entregar exactamente la demanda. Por ejemplo:
def demanda_clientes_rule(model, i):
    if i not in clients_dict.keys():
        return Constraint.Skip
    return sum(model.x[j, i, k] for k in model.K for j in model.I if j != i) >= 1
model.demanda_clientes = Constraint(model.I, rule=demanda_clientes_rule)

p_index_map = {'A':0, 'B':1, 'C':2}

def capacidad_demanda_rule(model, k, p):
    p_index = p_index_map[p]
    return model.W[k, p] == sum(
        clients_dict[i][p_index] * sum(model.x[i, j, k] for j in model.I if i != j)
        for i in model.C
    )

model.capacidad_demanda = Constraint(model.K, model.P, rule=capacidad_demanda_rule)



# Distancia Máxima
def distancia_maxima_rule(model, k):
    return sum(((model.distance[i,j]/1000) if vehicles_dict[k][0] != 'DR' else model.distance_air[i,j])*model.x[i,j,k]
               for (i,j) in model.PAIRS if i!=j) <= model.R[k]
model.distancia_maxima = Constraint(model.K, rule=distancia_maxima_rule)

# Definir s[k,d]
def s_definition_rule(model, k, d):
    return model.s[k,d] == sum(model.x[d,j,k] for j in model.I if d!=j)
model.s_definition = Constraint(model.K, model.D, rule=s_definition_rule)

def unica_salida_deposito_rule(model, k):
    return sum(model.s[k, d] for d in model.D) <= 1
model.unica_salida_deposito = Constraint(model.K, rule=unica_salida_deposito_rule)

# Linearización para cada producto
def linearizacion_producto_rule1(model, k, d, p):
    return model.W_s[k,d,p] <= model.W[k,p]
model.linearizacion_producto1 = Constraint(model.K, model.D, model.P, rule=linearizacion_producto_rule1)

def linearizacion_producto_rule2(model, k, d, p):
    return model.W_s[k,d,p] <= model.capacity_veh[k]*model.s[k,d]
model.linearizacion_producto2 = Constraint(model.K, model.D, model.P, rule=linearizacion_producto_rule2)

def linearizacion_producto_rule3(model, k, d, p):
    return model.W_s[k,d,p] >= model.W[k,p] - model.capacity_veh[k]*(1 - model.s[k,d])
model.linearizacion_producto3 = Constraint(model.K, model.D, model.P, rule=linearizacion_producto_rule3)

# Capacidad del depósito por producto
def capacidad_deposito_rule(model, d, p):
    return sum(model.W_s[k,d,p] for k in model.K) <= model.depot_capacity[d,p]
model.capacidad_deposito = Constraint(model.D, model.P, rule=capacidad_deposito_rule)

(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo


In [3]:
solver_name = "appsi_highs"
solver = SolverFactory(solver_name)
solver.options['parallel'] = 'on'
solver.options['threads'] = 6
solver.options['time_limit'] = 4000  # 1-hour time limit
solver.options['mip_rel_gap'] = 0.05  # 5% relative gap
result = solver.solve(model, tee=True)

Running HiGHS 1.8.1 (git hash: 4a7f24a): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [7e-01, 2e+02]
  Cost   [5e+02, 3e+05]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+03]
Presolving model
1234 rows, 2544 cols, 14596 nonzeros  0s
1205 rows, 1604 cols, 9662 nonzeros  0s
1034 rows, 1531 cols, 9082 nonzeros  0s

Solving MIP model with:
   1034 rows
   1531 cols (1307 binary, 0 integer, 18 implied int., 206 continuous)
   9082 nonzeros
MIP-Timing:        0.35 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol   

## Resultados


### Archivo CSV Rutas


In [4]:
import csv

# Ruta del archivo CSV de resultados
csv_results_path = "results/reporte_rutas.csv"

with open(csv_results_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Asumiendo que model.P es el conjunto de productos
    # Crear encabezados dinámicamente
    encabezados = ["Vehículo", "Ruta", "Distancia Total (km)"] + [f"Carga {p} (kg)" for p in model.P]
    writer.writerow(encabezados)

    # Generar rutas para cada vehículo
    for k in model.K:
        # Reconstruir la ruta ordenada
        ruta = []
        nodos_visitados = set()
        actual = None

        # Buscar el depósito de inicio
        for d in deposits_dict.keys():
            for j in model.I:
                if d != j and model.x[d, j, k].value > 0.5:
                    ruta.append(d)
                    actual = j
                    break
            if actual:
                break  # Salir si encontramos el nodo inicial

        # Seguir la ruta
        while actual and actual not in nodos_visitados:
            nodos_visitados.add(actual)
            ruta.append(actual)
            siguiente = None
            for j in model.I:
                if actual != j and model.x[actual, j, k].value > 0.5:
                    siguiente = j
                    break
            actual = siguiente

        # Agregar el depósito de fin si corresponde
        # (Opcional, dependerá si se tiene asegurado el regreso)
        for d in deposits_dict.keys():
            if actual == d:
                ruta.append(d)
                break

        # Calcular distancia total
        if len(ruta) > 1:
            distancia_total = sum(
                ((model.distance[i, j]/1000) if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j]) * model.x[i, j, k].value
                for i, j in zip(ruta[:-1], ruta[1:])
            )
        else:
            distancia_total = 0.0

        # Carga por producto
        cargas = [f"{model.W[k, p].value:.2f}" for p in model.P]

        # Escribir la fila al CSV
        writer.writerow([
            k,
            " -> ".join(ruta) if ruta else "Sin ruta",
            f"{distancia_total:.2f}"
        ] + cargas)

print(f"Archivo CSV de rutas generado: {csv_results_path}")


Archivo CSV de rutas generado: results/reporte_rutas.csv


### Reporte Resultados


In [5]:
results_path = "results/reporte_resultados.txt"

with open(results_path, "w") as file:
    # Escribir el costo total
    file.write("=== Resultados ===\n")
    file.write(f"Costo total operativo: {model.obj():.2f} COP\n\n")

    # Escribir los recorridos por vehículo en orden
    file.write("Recorridos asignados por vehículo:\n")
    for k in model.K:
        file.write(f"\nVehículo {k}:\n")
        ruta = []
        nodos_visitados = set()
        actual = None

        # Buscar el depósito de inicio
        for d in deposits_dict.keys():
            for j in model.I:
                if d != j and model.x[d, j, k].value > 0.5:
                    ruta.append(d)
                    actual = j
                    break
            if actual:
                break

        # Seguir la ruta
        while actual and actual not in nodos_visitados:
            nodos_visitados.add(actual)
            ruta.append(actual)
            siguiente = None
            for j in model.I:
                if actual != j and model.x[actual, j, k].value > 0.5:
                    siguiente = j
                    break
            actual = siguiente

        # Intentar agregar el depósito final si se conoce
        # (Esto es opcional, dependiendo de cómo se modela el regreso)
        for d in deposits_dict.keys():
            # Revisar si la ruta termina en un depósito
            if len(ruta) > 0 and ruta[-1] != d:
                # Si el vehículo regresa al depósito d, se agregará a la ruta
                # Ajustar si se desea forzar a imprimir el retorno exacto
                pass

        # Escribir la ruta ordenada
        if ruta:
            file.write(f"  Ruta: {' -> '.join(ruta)}\n")
        else:
            file.write("  No tiene asignaciones.\n")

    # Escribir distancias totales recorridas por vehículo
    file.write("\nDistancias totales recorridas:\n")
    for k in model.K:
        # Reconstruir ruta
        ruta = []
        nodos_visitados = set()
        actual = None
        for d in deposits_dict.keys():
            for j in model.I:
                if d != j and model.x[d, j, k].value > 0.5:
                    ruta.append(d)
                    actual = j
                    break
            if actual:
                break

        while actual and actual not in nodos_visitados:
            nodos_visitados.add(actual)
            ruta.append(actual)
            siguiente = None
            for j in model.I:
                if actual != j and model.x[actual, j, k].value > 0.5:
                    siguiente = j
                    break
            actual = siguiente

        # Calcular distancia total
        if len(ruta) > 1:
            distancia_total = 0.0
            for i, j in zip(ruta[:-1], ruta[1:]):
                if (i, j) in model.PAIRS:
                    dist = (model.distance[i, j]/1000 if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j])
                    distancia_total += dist * model.x[i, j, k].value
        else:
            distancia_total = 0.0

        file.write(f"  Vehículo {k}: {distancia_total:.2f} km\n")

    # Escribir pesos totales por vehículo y producto
    file.write("\nPesos totales transportados por vehículo y producto:\n")
    for k in model.K:
        file.write(f"  Vehículo {k}:\n")
        for p in model.P:
            peso = model.W[k, p].value
            file.write(f"    Producto {p}: {peso:.2f} kg\n")

print(f"Resultados escritos en el archivo: {results_path}")


Resultados escritos en el archivo: results/reporte_resultados.txt


In [6]:
import pandas as pd

# Cargar datos
routes_df = pd.read_csv('results/reporte_rutas.csv')

# Calcular costos

def calcular_costos_por_vehiculo():
    costos_por_vehiculo = {}

    for _, row in routes_df.iterrows():
        vehiculo = row['Vehículo']
        ruta = row['Ruta']
        if ruta == "Sin ruta":
            costos_por_vehiculo[vehiculo] = {'C_total': 0, 'C_tiempo': 0, 'C_carga': 0, 'C_mantenimiento': 0}
            continue

        tipo_vehiculo = vehicles_dict[vehiculo][0]
        tf = vehicles_parameters[tipo_vehiculo]['TF']
        tt = vehicles_parameters[tipo_vehiculo]['TT']
        cm = vehicles_parameters[tipo_vehiculo]['CM']
        vp = vehicles_parameters[tipo_vehiculo]['VP']

        C_total, C_tiempo, C_carga, C_mantenimiento = 0, 0, 0, cm

        nodos = ruta.split(" -> ")
        for i, j in zip(nodos[:-1], nodos[1:]):
            distancia = distance_dict[(i, j)] if tipo_vehiculo != "DR" else distance_air_dict[(i, j)]
            tiempo = (distance_dict[(i, j)] / 1000 / (vp / 60)) if tipo_vehiculo == "DR" else duration_dict[(i, j)] / 60

            # Calcular costos
            C_total += tf * (distancia / 1000)
            C_tiempo += tt * tiempo

        # Costo por peso transportado
        carga_a = row.get('Carga A (kg)', 0)
        carga_b = row.get('Carga B (kg)', 0)
        carga_c = row.get('Carga C (kg)', 0)
        C_carga += (carga_a + carga_b + carga_c) * 500

        costos_por_vehiculo[vehiculo] = {
            'C_total': C_total,
            'C_tiempo': C_tiempo,
            'C_carga': C_carga,
            'C_mantenimiento': C_mantenimiento
        }

    return costos_por_vehiculo

# Generar reporte
def generar_reporte():
    costos_por_vehiculo = calcular_costos_por_vehiculo()

    results_path = "results/reporte_desglosado.txt"
    with open(results_path, "w") as file:
        file.write("=== Reporte Desglosado ===\n")

        for vehiculo, costos in costos_por_vehiculo.items():
            file.write(f"Vehículo {vehiculo}:\n")
            file.write(f"  Costo por distancia: {costos['C_total']:.2f} COP\n")
            file.write(f"  Costo por tiempo: {costos['C_tiempo']:.2f} COP\n")
            file.write(f"  Costo por peso transportado: {costos['C_carga']:.2f} COP\n")
            file.write(f"  Costo de mantenimiento diario: {costos['C_mantenimiento']:.2f} COP\n")
            file.write(f"  Costo total: {sum(costos.values()):.2f} COP\n\n")

        file.write("=== Totales ===\n")
        total_costos = {k: sum(v[k] for v in costos_por_vehiculo.values()) for k in ['C_total', 'C_tiempo', 'C_carga', 'C_mantenimiento']}
        file.write(f"Costo total por distancia: {total_costos['C_total']:.2f} COP\n")
        file.write(f"Costo total por tiempo: {total_costos['C_tiempo']:.2f} COP\n")
        file.write(f"Costo total por peso transportado: {total_costos['C_carga']:.2f} COP\n")
        file.write(f"Costo total por mantenimiento: {total_costos['C_mantenimiento']:.2f} COP\n")
        file.write(f"Costo total operativo: {sum(total_costos.values()):.2f} COP\n")

    print(f"Reporte generado en {results_path}")

generar_reporte()

Reporte generado en results/reporte_desglosado.txt


### Mapa Interactivo


In [6]:
import folium
from folium.plugins import AntPath
import requests
from folium import FeatureGroup, LayerControl

# Crear un mapa centrado en una ubicación aproximada
centro_lat = deposits_df['Latitude'].mean()
centro_lon = deposits_df['Longitude'].mean()
mapa = folium.Map(location=[centro_lat, centro_lon], zoom_start=12)

# Asignar colores según el tipo de vehículo
tipo_colores = {
    "GS": "red",  # Gasoline cars
    "EV": "blue", # Electric vehicles
    "DR": "green" # Drones
}

colors = ['red', 'blue', 'green', 'purple', 'orange', 'black', 'darkred', 'darkblue']
vehicle_colors = {f"V{idx+1}": colors[idx % len(colors)] for idx in range(len(model.K))}

# Crear un grupo de características para cada vehículo
vehicle_groups = {k: FeatureGroup(name=f"Vehículo {k}") for k in model.K}

# Agregar marcadores para depósitos
for depot_id, data in deposits_dict.items():
    dep_id, lon, lat = data[0], data[1], data[2]
    # data = [int(row['DepotID']),row['Longitude'],row['Latitude'], int(row['Product-Type-A']), int(row['Product-Type-B']), int(row['Product-Type-C'])]
    # Ajustar popup si deseas mostrar las capacidades por producto (opcional)
    folium.Marker(
        location=[lat, lon],
        popup=f"Depósito {depot_id}",
        icon=folium.Icon(color="darkblue", icon="warehouse", prefix="fa")
    ).add_to(mapa)

# Agregar marcadores para clientes
for client_id, data in clients_dict.items():
    # data = [ProdA, ProdB, ProdC, Lon, Lat]
    folium.Marker(
        location=[data[4], data[3]],
        popup=f"Cliente {client_id}",
        icon=folium.Icon(color="green", icon="user", prefix="fa")
    ).add_to(mapa)

# Función para calcular ruta real con OSRM
def calcular_ruta_osrm(coord_inicio, coord_fin):
    try:
        url = f"http://router.project-osrm.org/route/v1/driving/{coord_inicio[1]},{coord_inicio[0]};{coord_fin[1]},{coord_fin[0]}"
        params = {"overview": "full", "geometries": "geojson"}
        response = requests.get(url, params=params)
        if response.status_code == 200:
            route = response.json()
            geometry = route["routes"][0]["geometry"]["coordinates"]
            return [(lat, lon) for lon, lat in geometry]
        else:
            print(f"Error OSRM: {response.status_code}")
            return [coord_inicio, coord_fin]
    except Exception as e:
        print(f"Error al calcular ruta OSRM: {e}")
        return [coord_inicio, coord_fin]

# Dibujar los recorridos de los vehículos con control de capas
for k in model.K:
    color = vehicle_colors[k]  # Asignar color único al vehículo
    grupo = vehicle_groups[k]

    for i in model.I:
        for j in model.I:
            if i != j and model.x[i, j, k].value > 0.5:  # Si hay un recorrido entre i y j
                # Obtener coordenadas de los nodos
                if i in clients_dict:
                    coord_i = (clients_dict[i][4], clients_dict[i][3])  # (lat, lon)
                else:
                    coord_i = (deposits_dict[i][2], deposits_dict[i][1])  # (lat, lon)
                if j in clients_dict:
                    coord_j = (clients_dict[j][4], clients_dict[j][3])  # (lat, lon)
                else:
                    coord_j = (deposits_dict[j][2], deposits_dict[j][1])  # (lat, lon)

                # Calcular la ruta (línea recta para drones, ruta OSRM para otros)
                ruta = [coord_i, coord_j] if vehicles_dict[k][0] == "DR" else calcular_ruta_osrm(coord_i, coord_j)

                # Preparar detalles de productos
                product_details = "<br>".join([f"Producto {p}: {model.W[k, p].value:.2f} kg" for p in model.P])

                # Dibujar la línea
                AntPath(
                    locations=ruta,
                    color=color,
                    weight=2.5,
                    opacity=1,
                    dash_array=[1, 20],
                    tooltip=(f"Vehículo {k}: {i} -> {j}<br>"
                             f"Distancia: {model.distance[i, j] / 1000:.2f} km<br>"
                             f"{product_details}")
                ).add_to(grupo)

    # Agregar el grupo al mapa
    grupo.add_to(mapa)

# Agregar el control de capas
LayerControl(collapsed=False).add_to(mapa)

# Guardar el mapa en un archivo HTML
mapa.save("results/mapa_rutas.html")
print("Mapa guardado como 'mapa_rutas.html'")


Mapa guardado como 'mapa_rutas.html'


### PDF de Reporte


In [7]:
!pip install fpdf
!pip install reportlab

In [8]:
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image, ListFlowable, ListItem
from reportlab.lib.units import cm
import re


def obtener_coordenadas(punto):
    if punto.startswith('D'):
        data = deposits_dict[punto]
        return (data[2], data[1])  # (lat, lon)
    elif punto.startswith('C'):
        data = clients_dict[punto]
        return (data[2], data[1])  # (lat, lon)
    return None

vehiculos_data = {}  # Guardar info general de rutas por vehiculo
distancias_data = {}
pesos_data = {}  # Aquí guardaremos datos por vehículo y producto
costo_total = None
vehiculos_encontrados = []  # Para mantener el orden en que se encuentran los vehículos

with open('results/reporte_resultados.txt', 'r', encoding='utf-8') as f:
    lines = f.read().strip().split('\n')

current_vehicle = None
parse_distancias = False
parse_pesos_productos = False
current_peso_veh = None

for line in lines:
    line = line.strip()
    # Costo total operativo
    if line.startswith("Costo total operativo:"):
        costo_match = re.search(r'Costo total operativo:\s+([\d.]+)', line)
        if costo_match:
            costo_total = float(costo_match.group(1))
    
    # Identificación de vehículos
    if line.startswith('Vehículo'):
        current_vehicle_match = re.search(r'(V\d+)', line)
        if current_vehicle_match:
            current_vehicle = current_vehicle_match.group(1)
            if current_vehicle not in vehiculos_encontrados:
                vehiculos_encontrados.append(current_vehicle)
            # Inicializar vehículo sin ruta si no existe
            if current_vehicle not in vehiculos_data:
                vehiculos_data[current_vehicle] = {
                    'ruta_completa': None,
                    'segmentos': []
                }
    
    # Parseo de ruta
    if line.startswith('Ruta:'):
        ruta_puntos = [p.strip() for p in line.replace('Ruta:', '').split('->')]
        coords = [obtener_coordenadas(p) for p in ruta_puntos if p]
        partes_url = []
        for c in coords:
            lat, lon = c
            partes_url.append(f"{lat},{lon}")
        gmaps_url = "https://www.google.com/maps/dir/" + "/".join(partes_url)
        
        # Guardar ruta completa
        vehiculos_data[current_vehicle]['ruta_completa'] = (ruta_puntos, gmaps_url)
        
        # Crear enlaces punto a punto
        segmentos = []
        for i in range(len(ruta_puntos)-1):
            inicio = ruta_puntos[i]
            fin = ruta_puntos[i+1]
            c_inicio = obtener_coordenadas(inicio)
            c_fin = obtener_coordenadas(fin)
            seg_url = f"https://www.google.com/maps/dir/{c_inicio[0]},{c_inicio[1]}/{c_fin[0]},{c_fin[1]}"
            segmentos.append((inicio, fin, seg_url))
        vehiculos_data[current_vehicle]['segmentos'] = segmentos
    
    # Parseo de distancias
    if "Distancias totales recorridas:" in line:
        parse_distancias = True
        continue
    if parse_distancias:
        if line.startswith("Vehículo"):
            dist_match = re.search(r'Vehículo\s+(V\d+):\s+([\d.]+)\s+km', line)
            if dist_match:
                dist_veh = dist_match.group(1)
                dist_val = float(dist_match.group(2))
                distancias_data[dist_veh] = dist_val
        else:
            # fin de sección distancias si no coincide
            if not line or not line.startswith("Vehículo"):
                parse_distancias = False
    
    # Parseo de pesos por producto
    if "Pesos totales transportados por vehículo y producto:" in line:
        parse_pesos_productos = True
        continue
    if parse_pesos_productos:
        if line.startswith("Vehículo"):
            peso_veh_match = re.search(r'Vehículo\s+(V\d+):', line)
            if peso_veh_match:
                current_peso_veh = peso_veh_match.group(1)
                if current_peso_veh not in pesos_data:
                    pesos_data[current_peso_veh] = {'A':0.0,'B':0.0,'C':0.0}
        elif "Producto A:" in line:
            a_match = re.search(r'Producto A:\s+([\d.-]+)\s+kg', line)
            if a_match and current_peso_veh:
                pesos_data[current_peso_veh]['A'] = float(a_match.group(1))
        elif "Producto B:" in line:
            b_match = re.search(r'Producto B:\s+([\d.-]+)\s+kg', line)
            if b_match and current_peso_veh:
                pesos_data[current_peso_veh]['B'] = float(b_match.group(1))
        elif "Producto C:" in line:
            c_match = re.search(r'Producto C:\s+([\d.-]+)\s+kg', line)
            if c_match and current_peso_veh:
                pesos_data[current_peso_veh]['C'] = float(c_match.group(1))

# Crear el PDF
doc = SimpleDocTemplate("results/informe.pdf", pagesize=A4)
styles = getSampleStyleSheet()
Story = []

# Agregar los logos
logo_uniandes = Image('data/logo_seneca.webp', width=4*cm, height=4*cm)
logo_otro = Image('data/LogoUnivLos_Andes02.png', width=4*cm, height=4*cm)
logos_table = Table([[logo_uniandes, logo_otro]], colWidths=[5*cm,5*cm])
logos_table.setStyle(TableStyle([
    ('ALIGN',(0,0),(-1,-1),'CENTER'),
    ('VALIGN',(0,0),(-1,-1),'MIDDLE')
]))
Story.append(logos_table)
Story.append(Spacer(1, 0.5*cm))

Story.append(Paragraph("Informe de Rutas", styles['Title']))
Story.append(Spacer(1, 0.5*cm))

# Sección Consultores
Story.append(Paragraph("Consultores:", styles['Heading2']))
Story.append(Spacer(1, 0.3*cm))
cons_list = ListFlowable([
    ListItem(Paragraph("Santiago Navarrete", styles['Normal'])),
    ListItem(Paragraph("Luis Ruiz", styles['Normal'])),
    ListItem(Paragraph("Andrea Lucia Galindo", styles['Normal']))
], bulletType='bullet', leftIndent=20)
Story.append(cons_list)
Story.append(Spacer(1, 0.5*cm))

# Costo total
if costo_total is not None:
    Story.append(Paragraph(f"Costo total operativo: {costo_total:.2f} COP", styles['Heading2']))
    Story.append(Spacer(1, 0.5*cm))

# Distancias totales
if distancias_data:
    Story.append(Paragraph("Distancias totales recorridas:", styles['Heading2']))
    Story.append(Spacer(1, 0.3*cm))
    tabla_dist = [["Vehículo", "Distancia (km)"]]
    for v, dist in distancias_data.items():
        tabla_dist.append([v, f"{dist:.2f} km"])
    t_dist = Table(tabla_dist, colWidths=[4*cm, 3*cm])
    t_dist.setStyle(TableStyle([
        ('BACKGROUND',(0,0),(-1,0), colors.gray),
        ('TEXTCOLOR',(0,0),(-1,0),colors.white),
        ('ALIGN',(0,0),(-1,-1),'LEFT'),
        ('GRID', (0,0), (-1,-1), 0.5, colors.black),
        ('VALIGN',(0,0),(-1,-1),'TOP')
    ]))
    Story.append(t_dist)
    Story.append(Spacer(1, 0.5*cm))

# Pesos totales por producto
if pesos_data:
    Story.append(Paragraph("Pesos totales transportados por vehículo y producto:", styles['Heading2']))
    Story.append(Spacer(1, 0.3*cm))
    tabla_pesos = [["Vehículo", "Producto A (kg)", "Producto B (kg)", "Producto C (kg)"]]
    for v, pdata in pesos_data.items():
        tabla_pesos.append([v, f"{pdata['A']:.2f}", f"{pdata['B']:.2f}", f"{pdata['C']:.2f}"])
    t_pesos = Table(tabla_pesos, colWidths=[3*cm, 3*cm, 3*cm, 3*cm])
    t_pesos.setStyle(TableStyle([
        ('BACKGROUND',(0,0),(-1,0), colors.gray),
        ('TEXTCOLOR',(0,0),(-1,0),colors.white),
        ('ALIGN',(0,0),(-1,-1),'LEFT'),
        ('GRID', (0,0), (-1,-1), 0.5, colors.black),
        ('VALIGN',(0,0),(-1,-1),'TOP')
    ]))
    Story.append(t_pesos)
    Story.append(Spacer(1, 0.5*cm))

# Rutas asignadas por vehículo
Story.append(Paragraph("Rutas asignadas por vehículo:", styles['Heading2']))
Story.append(Spacer(1, 0.3*cm))

for veh in vehiculos_encontrados:
    Story.append(Paragraph(f"Vehículo {veh}:", styles['Heading3']))
    Story.append(Spacer(1, 0.3*cm))
    datos_veh = vehiculos_data.get(veh, None)
    if datos_veh and datos_veh['ruta_completa']:
        # Ruta punto a punto
        segmentos = datos_veh['segmentos']
        if segmentos:
            Story.append(Paragraph("Detalle de ruta punto a punto:", styles['Heading4']))
            Story.append(Spacer(1, 0.3*cm))
            tabla_segmentos = [["Desde", "Hasta", "Link"]]
            for (inicio, fin, link_seg) in segmentos:
                link_par = Paragraph(f'<link href="{link_seg}">{link_seg}</link>', styles['Normal'])
                tabla_segmentos.append([inicio, fin, link_par])
            
            t_seg = Table(tabla_segmentos, colWidths=[3*cm, 3*cm, 11*cm])
            t_seg.setStyle(TableStyle([
                ('BACKGROUND',(0,0),(-1,0), colors.gray),
                ('TEXTCOLOR',(0,0),(-1,0),colors.white),
                ('ALIGN',(0,0),(-1,-1),'LEFT'),
                ('GRID', (0,0), (-1,-1), 0.5, colors.black),
                ('VALIGN',(0,0),(-1,-1),'TOP')
            ]))
            Story.append(t_seg)
            Story.append(Spacer(1, 0.5*cm))
        
        # Ruta completa
        (ruta_puntos, gmaps_url) = datos_veh['ruta_completa']
        Story.append(Paragraph("Ruta completa:", styles['Heading4']))
        Story.append(Spacer(1, 0.3*cm))
        link_par = Paragraph(f'<link href="{gmaps_url}">{gmaps_url}</link>', styles['Normal'])
        Story.append(Paragraph(f"({ ' -> '.join(ruta_puntos) })", styles['Normal']))
        Story.append(Spacer(1, 0.2*cm))
        Story.append(link_par)
        Story.append(Spacer(1, 0.5*cm))
    else:
        # Sin rutas
        Story.append(Paragraph("No tiene rutas.", styles['Normal']))
        Story.append(Spacer(1, 0.5*cm))

doc.build(Story)
